# adding context

In [5]:
import numpy as np
import pandas as pd
import concurrent.futures
import time

In [6]:
# show more columns and rows
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv("dfplusmin4.csv")

In [9]:
df.who.value_counts().head(152)

S0192        58247
S0084        45174
S0012        43024
S0041        34596
S0653        34147
S0525        32159
S0024        31524
S0439        29179
S0262        27069
S0058        26189
S0517        25448
S0198        24784
S0454        22873
S0144        22520
S0255        22136
S0152        21887
S0013        20792
S0008        20404
S0278        19058
S0115        18024
S0202        17908
S0037        17671
S0227        17255
S0179        17172
S0417        16050
S0336        15770
S0588        15138
S0018        14812
S0493        14663
S0529        14656
S0530        14346
S0655        13562
S0326        13223
S0654        13175
S0558        12673
S0618        12510
S0392        11963
S0423        11485
S0619        10666
S0589        10365
S0441        10356
S0261        10142
S0421         9868
S0328         9776
S0199         9683
S0208         9647
S0282         9206
S0167         9092
S0405         8945
S0337         8896
S0104         8872
S0416         8783
S0687       

In [4]:
n = 9  #chunk row size
contextList = [df[i:i+n] for i in range(0,df.shape[0],n)]

In [46]:
with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in contextList:
            if len(i) == 9:
                i["context"] = [-4,-3,-2,-1,0,1,2,3,4]
            else:
                i["context"] = "context"

<ipython-input-46-55ca0094611d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["context"] = [-4,-3,-2,-1,0,1,2,3,4]
<ipython-input-46-55ca0094611d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["context"] = "context"


In [47]:
oneSpeakerContext = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextList:
        temp = i.loc[i.who == i.iloc[4].who]
        oneSpeakerContext.append(temp)

In [150]:
#throws an error but thats ok, next cell takes care of it
oneSpOneN = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in oneSpeakerContext:
        temp1 =  i.loc[i.context == 0].n.array[0]
        if temp1 is not None:
            temp2 = i.loc[i.n == temp1]
            oneSpOneN.append(temp2)
        else:
            oneSpOneN.append(i)

In [130]:
#needed if above cell throws a warning
oneSpOneN.append(oneSpeakerContext[206766])

,class,conv,lemma,n,pos,trans,usas,who,whoConfidence,index,token,context
0,INTERJ,SZYV,mm,320,UH,overlap,Z4,S0432,high,11422610,mm,-4
1,INTERJ,SZYV,yeah,320,UH,overlap,Z4,S0432,high,11422611,yeah,-3
2,PRON,SZYV,we,320,PPIS2,overlap,Z8,S0432,high,11422612,we,-2
3,ADV,SZYV,probably,320,RR,overlap,A7,S0432,high,11422613,probably,-1
4,VERB,SZYV,should,320,VM,overlap,S6,S0432,high,11422614,should,0


In [158]:
len(oneSpOneN)

206767

In [186]:
test = oneSpOneN[0:100].copy()
iloc[i.loc[i.context == 0].index - 1]

In [151]:
# filter out all STOP class words, so full stop, question mark, exclamation mark.
# also filters out words before or after the stop, so that only the same sentence context remains
noStopContext = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in oneSpOneN:
        i.reset_index(drop=True, inplace=True)
        stopIndex = i.loc[i["class"] == "STOP"].index.tolist()
        if len(stopIndex) == 0:
            noStopContext.append(i)
        elif len(stopIndex) == 1:
            if stopIndex[0] < i.loc[i.context == 0].index.tolist()[0]:
                temp = i.iloc[stopIndex[0] + 1 :]
                noStopContext.append(temp)
            elif stopIndex[0] > i.loc[i.context == 0].index.tolist()[0]:
                temp = i.iloc[:stopIndex[0]]
                noStopContext.append(temp)
        elif len(stopIndex) == 2:
            if stopIndex[1] < i.loc[i.context == 0].index:
                temp = i.iloc[stopIndex[1] + 1 :]
                noStopContext.append(temp)
            elif stopIndex[0] > i.loc[i.context == 0].index:
                temp = i.iloc[:stopIndex[0]]
                noStopContext.append(temp)
            else:
                temp = i.iloc[stopIndex[0] + 1 : stopIndex[1]]
                noStopContext.append(temp)

In [152]:
#there was one df with "context" in all context cells, so i replaced them with the appropriate values
with concurrent.futures.ProcessPoolExecutor() as executor:
    for number, i in enumerate(noStopContext):
        if i.iloc[0].context == "context":
            i.context  = [-4,-3,-2,-1,0]
            print(i)

In [132]:
noStopContext

[   class  conv lemma   n    pos       trans  usas        who whoConfidence  \
 0   PRON  S23A     i  22  PPIS1  nonoverlap    Z8  UNKFEMALE           low   
 1   VERB  S23A  must  22     VM  nonoverlap    S6  UNKFEMALE           low   
 2   VERB  S23A    be  22    VBI  nonoverlap    A3  UNKFEMALE           low   
 3    ADJ  S23A  half  22     DB  nonoverlap  T1:3  UNKFEMALE           low   
 4    ART  S23A     a  22    AT1  nonoverlap  T1:3  UNKFEMALE           low   
 5  SUBST  S23A  hour  22   NNT1  nonoverlap  T1:3  UNKFEMALE           low   
 
    index token  context  
 0     78     I       -1  
 1     79  must        0  
 2     80    be        1  
 3     81  half        2  
 4     82    an        3  
 5     83  hour        4  ,
    class  conv  lemma   n  pos    trans  usas    who whoConfidence  index  \
 0    ADV  S23A  there  47   RL  overlap    Z4  S0032          high    240   
 1   PRON  S23A    you  47  PPY  overlap    Z4  S0032          high    241   
 2   VERB  S23A     g

The list called "noStopContext" now contains all modals with their proper contexts as dfs. From this the individual contexts can be read out.

# create context dfs

one where each line contains one word, one where each line contains the entire context

In [168]:
# use for saving the list
import pickle
with open("noStopContext.txt", "wb") as fp:
    pickle.dump(noStopContext, fp)

In [1]:
# use for loading the list
import pickle
with open("noStopContext.txt", "rb") as fp:
    noStopContext = pickle.load(fp)

In [2]:
# creates deep copy of the context list created until here so that the longer operations don't have to be rerun
# whenever the list is changed
# rerun before creating new context df
import copy
contextClean = copy.deepcopy(noStopContext)

In [6]:
len(contextClean)

206757

## 2R0L context

In [7]:
# create list of dfs that only contain the context 2 words to the right of the modal
context2R = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextClean:
        if i.context.tolist()[-1] < 2:
            continue
        else:
            modalIndex = i.loc[i.context == 0].index[0]
            temp = i.iloc[modalIndex : modalIndex + 3]
            context2R.append(temp)

In [4]:
context2R

NameError: name 'context2R' is not defined

In [8]:
# make list to list of lists rather than list of dfs for easier transformation to one large df.
context2Rlist = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in context2R:
            for j in range(i.shape[0]):
                context2Rlist.append(i.iloc[j].tolist())

KeyboardInterrupt: 

In [ ]:
context2R0Ldf = pd.DataFrame(context2Rlist, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [417]:
context2R0Ldf.to_csv("context2R0Ldf.csv", index=False)

## single line 2R0L

In [9]:
fragmentList = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for j in context2R:
        fragment = ""
        for i in j.token.astype("str").tolist():
            fragment += i + " "
        fragmentList.append(fragment)

singleCellContext = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in context2R:
        singleCellContext.append(i.iloc[2:3,:])

j = 0
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in singleCellContext:
        i.token = fragmentList[j]
        j += 1

singleCellContextList = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in singleCellContext:
            for j in range(i.shape[0]):
                singleCellContextList.append(i.iloc[j].tolist())

singleCell2R0Ldf = pd.DataFrame(singleCellContextList, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

C:\Users\Clemens\anaconda3\envs\Mouton\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
singleCell2R0Ldf.token.value_counts().head(30)

'll have to             1426
'd have to               806
'd like to               684
'll have a               638
would you like           623
would have been          614
can I have               558
'd be like               539
would be a               518
can't remember what      488
must have been           471
can do it                454
'll be like              394
can have a               378
would 've been           378
can do that              366
'll I 'll                359
'll do it                359
might as well            351
might be a               346
'll tell you             343
'll be fine              329
can't do it              327
'll be a                 309
would like to            296
would be like            287
would be nice            280
can't do that            279
'd love to               272
might have to            258
Name: token, dtype: int64

In [11]:
singleCell2R0Ldf.to_csv("singleCell2R0Ldf.csv", index=False)

## 0R2L context

In [7]:
# create list of dfs that only contain the context 2 word to the left of the modal
context0R2L = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextClean:
        #check if df doesnt have enough words to the left
        if i.context.tolist()[0] > -2:
            continue
        else:
            modalIndex = i.loc[i.context == 0].index[0]
            temp = i.iloc[modalIndex - 2 : modalIndex + 1]
            context0R2L.append(temp)

# make list to list of lists rather than list of dfs for easier transformation to one large df.
context0R2Llist = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in context0R2L:
            for j in range(i.shape[0]):
                context0R2Llist.append(i.iloc[j].tolist())

context0R2Ldf = pd.DataFrame(context0R2Llist, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [8]:
context0R2L[1]

,class,conv,lemma,n,pos,trans,usas,who,whoConfidence,index,token,context
2,VERB,S23A,assume,60,VVD,overlap,X2:1,S0032,high,372,assumed,-2
3,PRON,S23A,it,60,PPH1,overlap,Z8,S0032,high,373,it,-1
4,VERB,S23A,would,60,VM,overlap,A7,S0032,high,374,would,0


In [9]:
#single cell context
fragmentList = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for j in context0R2L:
        fragment = ""
        for i in j.token.astype("str").tolist():
            fragment += i + " "
        fragmentList.append(fragment)

singleCellContext = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in context0R2L:
        singleCellContext.append(i.iloc[2:3,:])

j = 0
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in singleCellContext:
        i.token = fragmentList[j]
        j += 1

singleCellContextList = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in singleCellContext:
            for j in range(i.shape[0]):
                singleCellContextList.append(i.iloc[j].tolist())

singleCell0R2Ldf = pd.DataFrame(singleCellContextList, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

C:\Users\Clemens\anaconda3\envs\Mouton\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
singleCell0R2Ldf.to_csv("singleCell0R2Ldf.csv", index=False)

In [10]:
singleCell0R2Ldf.token.value_counts().head(20)

and you can      884
and I 'll        820
so you can       686
that you can     676
yeah I 'll       587
well I 'll       530
if you can       506
so I 'll         505
then I 'll       492
but you can      483
but I can't      430
then you can     430
yeah you can     429
so I can         417
if I can         408
and I 'd         381
well you can     371
oh I 'll         366
'll I 'll        361
and we 'll       358
Name: token, dtype: int64

## 3R context

In [14]:
# create list of dfs that only contain the context 2 words to the right of the modal
context3R = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextClean:
        if i.context.tolist()[-1] < 3:
            continue
        else:
            modalIndex = i.loc[i.context == 0].index[0]
            temp = i.iloc[modalIndex : modalIndex + 4]
            context3R.append(temp)

# make list to list of lists rather than list of dfs for easier transformation to one large df.
context3Rlist = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in context3R:
            for j in range(i.shape[0]):
                context3Rlist.append(i.iloc[j].tolist())

context3R0Ldf = pd.DataFrame(context3Rlist, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [424]:
context3R0Ldf.to_csv("context3R0Ldf.csv", index=False)

### create df with 3R context in single cell rather than line for each word

In [17]:
fragmentList = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for j in context3R:
        fragment = ""
        for i in j.token.astype("str").tolist():
            fragment += i + " "
        fragmentList.append(fragment)

In [23]:
singleCellContext = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in context3R:
        singleCellContext.append(i.iloc[0:1,:])

In [26]:
j = 0
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in singleCellContext:
        i.token = fragmentList[j]
        j += 1

C:\Users\Clemens\anaconda3\envs\Mouton\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [33]:
singleCellContextList = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in singleCellContext:
            for j in range(i.shape[0]):
                singleCellContextList.append(i.iloc[j].tolist())

singleCell3R0Ldf = pd.DataFrame(singleCellContextList, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [35]:
singleCell3R0Ldf.token.value_counts().head(20)

'll be able to             230
won't be able to           203
wouldn't be able to        201
can I have a               199
would you like to          194
can't remember what it     179
might be able to           165
'll have a look            160
'll tell you what          134
should be able to          129
'll have to go             120
might be a bit             117
'd like to go              112
'll just have to            98
'll have to get             94
'd be able to               93
would you like a            89
would be a good             87
'll have to do              86
'd be like oh               86
Name: token, dtype: int64

In [88]:
singleCell3R0Ldf.to_csv("singleCell3R0Ldf.csv", index=False)

## 2R2L context

In [79]:
# create list of dfs that only contain the context 2 words to the right and two words to the left of the modal
context2R2L = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextClean:
        #check if df doesnt have enough words to the right
        if i.context.tolist()[-1] < 2:
            continue
        #check if df doesnt have enough words to the left
        elif i.context.tolist()[0] > -2:
            continue
        else:
            modalIndex = i.loc[i.context == 0].index[0]
            temp = i.iloc[modalIndex - 2 : modalIndex + 3]
            context2R2L.append(temp)

# make list to list of lists rather than list of dfs for easier transformation to one large df.
context2R2Llist = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in context2R2L:
            for j in range(i.shape[0]):
                context2R2Llist.append(i.iloc[j].tolist())

context2R2Ldf = pd.DataFrame(context2R2Llist, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [89]:
context2R2Ldf.to_csv("context2R2Ldf.csv", index=False)

### create 2R2L df with tokens in single cell

In [84]:
fragmentList = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for j in context2R2L:
        fragment = ""
        for i in j.token.astype("str").tolist():
            fragment += i + " "
        fragmentList.append(fragment)

singleCellContext = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in context2R2L:
        singleCellContext.append(i.iloc[2:3,:])

j = 0
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in singleCellContext:
        i.token = fragmentList[j]
        j += 1

singleCellContextList = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in singleCellContext:
            for j in range(i.shape[0]):
                singleCellContextList.append(i.iloc[j].tolist())

singleCell2R2Ldf = pd.DataFrame(singleCellContextList, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

C:\Users\Clemens\anaconda3\envs\Mouton\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [85]:
singleCell2R2Ldf.token.value_counts().head(20)

yeah I 'll have a             40
yeah I 'll have to            38
but I can't remember what     37
so I 'll have to              36
yeah I 'll I 'll              31
and I 'll tell you            29
no you can't do that          25
but you 'd have to            25
and I 'd be like              25
think I 'd like to            24
yeah I 'd like to             24
and I 'll be like             24
yeah we 'll have to           24
well we 'll have to           24
and I can't remember what     23
but I 'd like to              23
yeah I 'd love to             23
well I 'll have to            23
sure it 'll be fine           23
well I 'll I 'll              23
Name: token, dtype: int64

In [86]:
singleCell2R2Ldf.to_csv("singleCell2R2Ldf.csv", index=False)

# 1R0L context

In [177]:
# create list of dfs that only contain the context 2 words to the right and two words to the left of the modal
context1R0L = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextClean:
        i.reset_index(drop=True, inplace=True)
        #check if df doesnt have enough words to the right
        if i.context.tolist()[-1] < 1:
            continue
        else:
            modalIndex = i.loc[i.context == 0].index[0]
            temp = i.iloc[modalIndex: modalIndex + 2]
            context1R0L.append(temp)

# make list to list of lists rather than list of dfs for easier transformation to one large df.
context1R0Llist = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in context1R0L:
            for j in range(i.shape[0]):
                context1R0Llist.append(i.iloc[j].tolist())

context1R0Ldf = pd.DataFrame(context1R0Llist, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [139]:
context1R0Ldf.to_csv("contex1R0Ldf.csv", index=False)

## 1R0L single line context df

In [178]:
fragmentList = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for j in context1R0L:
        fragment = ""
        for i in j.token.astype("str").tolist():
            fragment += i + " "
        fragmentList.append(fragment)

singleCellContext = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in context1R0L:
        singleCellContext.append(i.iloc[0:1,:])

j = 0
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in singleCellContext:
        i.token = fragmentList[j]
        j += 1

singleCellContextList = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in singleCellContext:
            for j in range(i.shape[0]):
                singleCellContextList.append(i.iloc[j].tolist())

singleCell1R0Ldf = pd.DataFrame(singleCellContextList, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

C:\Users\Clemens\anaconda3\envs\Mouton\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [179]:
singleCell1R0Ldf.token.value_counts().head(20)

'll be             6131
would be           5602
'd be              3929
can you            3872
'll have           3340
would you          2744
might be           2694
would have         2456
can I              2428
can do             2350
can get            2204
'll just           2094
can't remember     2080
will be            2040
'd have            2010
could be           1929
should be          1903
must be            1654
could have         1496
could do           1486
Name: token, dtype: int64

In [180]:
singleCell1R0Ldf.to_csv("singleCell1R0Ldf.csv", index=False)

# 0R1L context

In [160]:
# create list of dfs that only contain the context 1 word to the left of the modal
context0R1L = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in contextClean:
        #check if df doesnt have enough words to the left
        if i.context.tolist()[0] > -1:
            continue
        else:
            modalIndex = i.loc[i.context == 0].index[0]
            temp = i.iloc[modalIndex - 1 : modalIndex + 1]
            context0R1L.append(temp)

# make list to list of lists rather than list of dfs for easier transformation to one large df.
context0R1Llist = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in context0R1L:
            for j in range(i.shape[0]):
                context0R1Llist.append(i.iloc[j].tolist())

context0R1Ldf = pd.DataFrame(context0R1Llist, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

In [161]:
context0R1Ldf.head(200)

,class,conv,lemma,n,pos,trans,usas,who,whoConfidence,index,token,context
0,PRON,S23A,i,22,PPIS1,nonoverlap,Z8,UNKFEMALE,low,78,I,-1
1,VERB,S23A,must,22,VM,nonoverlap,S6,UNKFEMALE,low,79,must,0
2,VERB,S23A,go,47,VV0,overlap,Z4,S0032,high,242,go,-1
3,VERB,S23A,would,47,VM,overlap,A7,S0032,high,243,would,0
4,PRON,S23A,it,57,PPH1,nonoverlap,Z8,S0032,high,300,it,-1
5,VERB,S23A,should,57,VM,nonoverlap,S6,S0032,high,301,should,0
6,PRON,S23A,it,60,PPH1,overlap,Z8,S0032,high,373,it,-1
7,VERB,S23A,would,60,VM,overlap,A7,S0032,high,374,would,0
8,INTERJ,S23A,er,63,UH,overlap,Z4,S0094,high,401,er,-1
9,VERB,S23A,shall,63,VM,overlap,T1:1:3,S0094,high,402,shall,0


In [162]:
context0R1Ldf.to_csv("contex0R1Ldf.csv", index=False)

### 0R1L single line df

In [163]:
fragmentList = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for j in context0R1L:
        fragment = ""
        for i in j.token.astype("str").tolist():
            fragment += i + " "
        fragmentList.append(fragment)

singleCellContext = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in context0R1L:
        singleCellContext.append(i.iloc[1:2,:])

j = 0
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in singleCellContext:
        i.token = fragmentList[j]
        j += 1

singleCellContextList = []

with concurrent.futures.ProcessPoolExecutor() as executor:
        for i in singleCellContext:
            for j in range(i.shape[0]):
                singleCellContextList.append(i.iloc[j].tolist())

singleCell0R1Ldf = pd.DataFrame(singleCellContextList, columns=['class', 'conv', 'lemma', 'n', 'pos', 'trans', 'usas', 'who',
       'whoConfidence', 'index', 'token', 'context'])

C:\Users\Clemens\anaconda3\envs\Mouton\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [164]:
singleCell0R1Ldf.head(200)

,class,conv,lemma,n,pos,trans,usas,who,whoConfidence,index,token,context
0,VERB,S23A,must,22,VM,nonoverlap,S6,UNKFEMALE,low,79,I must,0
1,VERB,S23A,would,47,VM,overlap,A7,S0032,high,243,go would,0
2,VERB,S23A,should,57,VM,nonoverlap,S6,S0032,high,301,it should,0
3,VERB,S23A,would,60,VM,overlap,A7,S0032,high,374,it would,0
4,VERB,S23A,shall,63,VM,overlap,T1:1:3,S0094,high,402,er shall,0
5,VERB,S23A,could,76,VM,nonoverlap,A7,S0094,high,499,you could,0
6,VERB,S23A,could,76,VM,nonoverlap,A7,S0094,high,502,we could,0
7,VERB,S23A,will,82,VM,nonoverlap,T1:1:3,S0094,high,550,I 'll,0
8,VERB,S23A,will,82,VM,nonoverlap,T1:1:3,S0094,high,556,I 'll,0
9,VERB,S23A,can,101,VM,nonoverlap,A7,S0032,high,710,but can,0


In [166]:
singleCell0R1Ldf.token.value_counts().head(30)

you can        13745
I 'll          13028
I can't         7234
I 'd            6860
I can           6775
you could       4987
we 'll          4961
you can't       4673
I would         4292
you 'd          3953
it would        3768
I could         3538
you 'll         3081
it 'll          2850
we can          2694
that would      2527
they 'll        2418
I might         2109
I wouldn't      2074
we could        2056
you should      2016
they can        1742
you would       1690
they 'd         1596
it might        1594
he 'll          1588
I couldn't      1560
I will          1487
you might       1476
I should        1436
Name: token, dtype: int64

In [171]:
singleCell0R1Ldf.to_csv("singleCell0R1Ldf.csv", index=False)